In [31]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets  #torchvision/torchtext/torchAudio 都包含数据集  
from torchvision.transforms import ToTensor

In [32]:
train_data = datasets.FashionMNIST(root="../datasets",
train= True,transform= ToTensor(),
download=True)
test_data = datasets.FashionMNIST(root="../datasets",
train= False,transform= ToTensor(),
download=True)
#TOTENSOR将目标值缩放到了我们【0，1】之间

In [33]:
"""
数据加载器结合了  数据集 和  采样器，并提供对给定数据集的可迭代对象。 
支持映射样式
和可迭代样式的数据集，
具有单进程或多进程加载、
自定义加载顺序
以及可选的自动批处理（排序规则）
和内存固定
sampler和shuffle不能并存,sampler的可以自定义采样策略
batch_sampler 和batch_size, shuffle, sampler, and drop_last不能并存


设计到数据集取样的东西可以在DataLoader中去设置
"""

train_dataload = DataLoader(train_data,batch_size=64,shuffle = True,num_workers = 4,drop_last = True)
test_dataload = DataLoader(test_data,batch_size=64,shuffle = True,num_workers = 4,drop_last = True)

"""
创建了一个迭代器，每个迭代的元素就有64个
"""

for x,y in train_dataload:
    print(x.shape)
    print(y.shape)
    break
# print(len(train_dataload))

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [34]:
"""
创建一个模型，并且在创建了模型以后将其直接加载到cuda上面去
"""
from turtle import forward


device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print("decive = {}".format(device))

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.liner_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self,x):
        x = self.flatten(x)
        x = self.liner_relu_stack(x)
        return x

predict_model = MLP()
predict_model.to(device)

decive = cuda


MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (liner_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [35]:
import torch.optim as optim
loss = nn.CrossEntropyLoss()
optims = optim.Adam(predict_model.parameters(),lr=0.001)

In [41]:
def train(dataloader : DataLoader,model : MLP,loss_F,optim):
    model.train()
    for batch, (x,y) in enumerate(dataloader):
        x,y = x.to(device),y.to(device)
        predict = model(x)
        optim.zero_grad()
        loss = loss_F(predict,y)
        loss.backward()
        optim.step()
        if batch % 500 ==0 :
            loss = loss.item()
            print("第{}批 ： loss = {}".format(batch,loss))

def test(dataloader : DataLoader,model : MLP,loss_F):
    model.eval()
    current = 0
    batch_all = 0
    with torch.no_grad():
        for batch, (x,y) in enumerate(dataloader):
            x,y = x.to(device),y.to(device)
            predict = model(x) #predict的形状是64✖10的
            loss = loss_F(predict,y)
            # if batch % 10 == 0:
            #     print(f"test_loss{loss}")
            current += (predict.argmax(dim = 1) == y).float().sum().item()
            batch_all += 64
    print(f"总的准确度{current/batch_all}")
    

In [42]:
for i in range(10):
    print(f"{i}/10 :")
    train(train_dataload,predict_model,loss,optims)
    test(train_dataload,predict_model,loss)
print("done!")

0/10 :
第0批 ： loss = 0.3828061819076538
第500批 ： loss = 0.34215041995048523
总的准确度0.8910919156883671
1/10 :
第0批 ： loss = 0.37918615341186523
第500批 ： loss = 0.22067591547966003
总的准确度0.8905749733191035
2/10 :
第0批 ： loss = 0.2693765163421631
第500批 ： loss = 0.4474758803844452
总的准确度0.8990128068303095
3/10 :
第0批 ： loss = 0.1845795065164566
第500批 ： loss = 0.266044020652771
总的准确度0.906049893276414
4/10 :
第0批 ： loss = 0.31556612253189087
第500批 ： loss = 0.2559833824634552
总的准确度0.9107190501600854
5/10 :
第0批 ： loss = 0.33894336223602295
第500批 ： loss = 0.31940582394599915
总的准确度0.9140875133404482
6/10 :
第0批 ： loss = 0.14086760580539703
第500批 ： loss = 0.2193586528301239
总的准确度0.9173392475987193
7/10 :
第0批 ： loss = 0.2069055140018463
第500批 ： loss = 0.24145254492759705
总的准确度0.9259938633938101
8/10 :
第0批 ： loss = 0.27299946546554565
第500批 ： loss = 0.1824720948934555
总的准确度0.9167556029882604
9/10 :
第0批 ： loss = 0.29165220260620117
第500批 ： loss = 0.22651247680187225
总的准确度0.932130469583778
done!


In [43]:
torch.save(predict_model.state_dict(), "predict_model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
